#Importing important libraries

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy, AUC
from tensorflow.keras.applications import InceptionV3  # Import InceptionV3 as GoogleNet is not directly available
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**CNN Implementation**

1. **GoogleNet Model**

In [ ]:
# Define paths
input_folder = '/content/drive/MyDrive/yolo_dataset'  # Change this to your input folder path
output_folder = os.path.join(input_folder, 'new_output')
obj_names_path = '/content/drive/MyDrive/yolo_dataset/obj.names'  # Change this to your obj_names file path

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to read class labels from obj_names file
def read_class_labels(obj_names_path):
    with open(obj_names_path, 'r') as file:
        class_labels = file.read().splitlines()
    # Ensure "saloon cars" is first and "motorcycles" is second
    if class_labels[0] != "saloon cars":
        class_labels[0], class_labels[1] = class_labels[1], class_labels[0]
    return class_labels

# Function to draw bounding boxes and labels on the image
def draw_bounding_boxes(image_path, annotation_path, class_labels):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(annotation_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
            class_id = int(class_id)
            x_center *= width
            y_center *= height
            box_width *= width
            box_height *= height

            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))
            x_max = int(x_center + (box_width / 2))
            y_max = int(x_center + (box_width / 2))

            # Draw the rectangle on the image
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

            # Put class label text on the image
            label = class_labels[class_id]
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return image

# Read class labels from obj_names file
class_labels = read_class_labels(obj_names_path)

# Process each image and its corresponding annotation
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            annotated_image = draw_bounding_boxes(image_path, annotation_path, class_labels)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, annotated_image)
            print(f'Saved annotated image: {output_path}')

print('Processing complete.')

# Create separate folders for each class if they don't exist
saloon_cars_folder = os.path.join(output_folder, 'saloon_cars')
motorcycles_folder = os.path.join(output_folder, 'motorcycles')

if not os.path.exists(saloon_cars_folder):
    os.makedirs(saloon_cars_folder)

if not os.path.exists(motorcycles_folder):
    os.makedirs(motorcycles_folder)

# Function to crop and save image patches
def crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, count):
    cropped_image = image[y_min:y_max, x_min:x_max]
    if class_id == 0:
        output_path = os.path.join(saloon_cars_folder, f'saloon_cars_{count}_{filename}')
    elif class_id == 1:
        output_path = os.path.join(motorcycles_folder, f'motorcycles_{count}_{filename}')
    cv2.imwrite(output_path, cropped_image)
    print(f'Saved cropped image: {output_path}')

# Initialize a counter for each class to ensure unique filenames
saloon_cars_count = 0
motorcycles_count = 0

# Process each image and its corresponding annotation to crop and save patches
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            with open(annotation_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
                    class_id = int(class_id)
                    x_center *= width
                    y_center *= height
                    box_width *= width
                    box_height *= height

                    x_min = int(x_center - (box_width / 2))
                    y_min = int(y_center - (box_height / 2))
                    x_max = int(x_center + (box_width / 2))
                    y_max = int(x_center + (box_width / 2))

                    # Crop and save the image patch
                    if class_id == 0:
                        saloon_cars_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, saloon_cars_count)
                    elif class_id == 1:
                        motorcycles_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, motorcycles_count)

print('Cropping and saving complete.')

# Load and preprocess the dataset
dataset_new = image_dataset_from_directory(
    '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images',
    batch_size=32,
    image_size=(224, 224),
    label_mode='int'
)

scaled_data = dataset_new.map(lambda x, y: (preprocess_input(x), y))

# Split the dataset into train and test sets
dataset_size = tf.data.experimental.cardinality(scaled_data).numpy()
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train = scaled_data.take(train_size)
test = scaled_data.skip(train_size).take(test_size)

# Define the GoogleNet (Inception V1) model architecture using InceptionV3 as base
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of the base model
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model with limited batches
hist = model.fit(train, validation_data=test, epochs=5, steps_per_epoch=15, validation_steps=15)

# Model evaluation metrics
y_true = []
y_pred = []

# Evaluate the model
for batch in test.as_numpy_iterator():
    X, y = batch
    y_pred_batch = model.predict(X)
    y_true.extend(y)
    y_pred.extend(np.argmax(y_pred_batch, axis=1))

# Convert y_true and y_pred to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(f'Confusion Matrix:\n{cm}')

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()

# Calculate and print AUC, precision, recall, and F1 score
auc = roc_auc_score(y_true, y_pred, multi_class='ovo')
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
classification_rep = classification_report(y_true, y_pred, target_names=class_labels)

print(f'AUC: {auc}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Classification Report:\n{classification_rep}')

# Plot training and validation accuracy
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='blue', label='val_accuracy')
fig.suptitle('ACCURACY', fontsize)
fig.suptitle('ACCURACY', fontsize=20)
plt.legend(loc='upper right')
plt.show()

# Plot training and validation loss
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='blue', label='val_loss')
fig.suptitle('LOSS', fontsize=20)
plt.legend(loc='upper right')
plt.show()

2. **LeNet-5**

In [ ]:
# Define paths
input_folder = '/content/drive/MyDrive/yolo_dataset'  # Change this to your input folder path
output_folder = os.path.join(input_folder, 'new_output')
obj_names_path = '/content/drive/MyDrive/yolo_dataset/obj.names'  # Change this to your obj_names file path

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to read class labels from obj_names file
def read_class_labels(obj_names_path):
    with open(obj_names_path, 'r') as file:
        class_labels = file.read().splitlines()
    # Ensure "saloon cars" is first and "motorcycles" is second
    if class_labels[0] != "saloon cars":
        class_labels[0], class_labels[1] = class_labels[1], class_labels[0]
    return class_labels

# Function to draw bounding boxes and labels on the image
def draw_bounding_boxes(image_path, annotation_path, class_labels):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(annotation_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
            class_id = int(class_id)
            x_center *= width
            y_center *= height
            box_width *= width
            box_height *= height

            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))
            x_max = int(x_center + (box_width / 2))
            y_max = int(y_center + (box_width / 2))

            # Draw the rectangle on the image
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

            # Put class label text on the image
            label = class_labels[class_id]
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return image

# Read class labels from obj_names file
class_labels = read_class_labels(obj_names_path)

# Process each image and its corresponding annotation
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            annotated_image = draw_bounding_boxes(image_path, annotation_path, class_labels)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, annotated_image)
            print(f'Saved annotated image: {output_path}')

print('Processing complete.')

# Create separate folders for each class if they don't exist
saloon_cars_folder = os.path.join(output_folder, 'saloon_cars')
motorcycles_folder = os.path.join(output_folder, 'motorcycles')

if not os.path.exists(saloon_cars_folder):
    os.makedirs(saloon_cars_folder)

if not os.path.exists(motorcycles_folder):
    os.makedirs(motorcycles_folder)

# Function to crop and save image patches
def crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, count):
    cropped_image = image[y_min:y_max, x_min:x_max]
    if class_id == 0:
        output_path = os.path.join(saloon_cars_folder, f'saloon_cars_{count}_{filename}')
    elif class_id == 1:
        output_path = os.path.join(motorcycles_folder, f'motorcycles_{count}_{filename}')
    cv2.imwrite(output_path, cropped_image)
    print(f'Saved cropped image: {output_path}')

# Initialize a counter for each class to ensure unique filenames
saloon_cars_count = 0
motorcycles_count = 0

# Process each image and its corresponding annotation to crop and save patches
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            with open(annotation_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
                    class_id = int(class_id)
                    x_center *= width
                    y_center *= height
                    box_width *= width
                    box_height *= height

                    x_min = int(x_center - (box_width / 2))
                    y_min = int(y_center - (box_height / 2))

                    x_max = int(x_center + (box_width / 2))
                    y_max = int(x_center + (box_width / 2))

                    # Crop and save the image patch
                    if class_id == 0:
                        saloon_cars_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, saloon_cars_count)
                    elif class_id == 1:
                        motorcycles_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, motorcycles_count)

print('Cropping and saving complete.')

# Load and preprocess the dataset
dataset_new = image_dataset_from_directory(
    '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images',
    batch_size=32,
    image_size=(32, 32),  # LeNet-5 works with 32x32 images
    label_mode='int'
)

scaled_data = dataset_new.map(lambda x, y: (x / 255.0, y))

# Split the dataset into train and test sets
dataset_size = tf.data.experimental.cardinality(scaled_data).numpy()
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train = scaled_data.take(train_size)
test = scaled_data.skip(train_size).take(test_size)

# Define the LeNet-5 model architecture
model = Sequential([
    Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
hist = model.fit(train, validation_data=test, epochs=5)

# Model evaluation metrics
y_true = []
y_pred = []

# Evaluate the model
for batch in test.as_numpy_iterator():
    X, y = batch
    y_pred_batch = model.predict(X)
    y_true.extend(y)
    y_pred.extend(np.argmax(y_pred_batch, axis=1))

# Convert y_true and y_pred to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(f'Confusion Matrix:\n{cm}')

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()

# Calculate and print AUC, precision, recall, and F1 score
auc = roc_auc_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
classification_rep = classification_report(y_true, y_pred, target_names=class_labels)

print(f'AUC: {auc}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Classification Report:\n{classification_rep}')

# Plot training and validation accuracy
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='blue', label='val_accuracy')
fig.suptitle('ACCURACY', fontsize=20)
plt.legend(loc='upper right')
plt.show()

# Plot training and validation loss
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='blue', label='val_loss')
fig.suptitle('LOSS', fontsize=20)
plt.legend(loc='upper right')
plt.show()



3. **VGG-16**

In [ ]:
# Define paths
input_folder = '/content/drive/MyDrive/yolo_dataset'  # Change this to your input folder path
output_folder = os.path.join(input_folder, 'new_output')
obj_names_path = '/content/drive/MyDrive/yolo_dataset/obj.names'  # Change this to your obj_names file path

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to read class labels from obj_names file
def read_class_labels(obj_names_path):
    with open(obj_names_path, 'r') as file:
        class_labels = file.read().splitlines()
    # Ensure "saloon cars" is first and "motorcycles" is second
    if class_labels[0] != "saloon cars":
        class_labels[0], class_labels[1] = class_labels[1], class_labels[0]
    return class_labels

# Function to draw bounding boxes and labels on the image
def draw_bounding_boxes(image_path, annotation_path, class_labels):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    with open(annotation_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
            class_id = int(class_id)
            x_center *= width
            y_center *= height
            box_width *= width
            box_height *= height

            x_min = int(x_center - (box_width / 2))
            y_min = int(x_center - (box_width / 2))
            x_max = int(x_center + (box_width / 2))
            y_max = int(y_center + (box_width / 2))

            # Draw the rectangle on the image
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

            # Put class label text on the image
            label = class_labels[class_id]
            cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return image

# Read class labels from obj_names file
class_labels = read_class_labels(obj_names_path)

# Process each image and its corresponding annotation
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            annotated_image = draw_bounding_boxes(image_path, annotation_path, class_labels)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, annotated_image)
            print(f'Saved annotated image: {output_path}')

print('Processing complete.')

# Create separate folders for each class if they don't exist
saloon_cars_folder = os.path.join(output_folder, 'saloon_cars')
motorcycles_folder = os.path.join(output_folder, 'motorcycles')

if not os.path.exists(saloon_cars_folder):
    os.makedirs(saloon_cars_folder)

if not os.path.exists(motorcycles_folder):
    os.makedirs(motorcycles_folder)

# Function to crop and save image patches
def crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, count):
    cropped_image = image[y_min:y_max, x_min:x_max]
    if class_id == 0:
        output_path = os.path.join(saloon_cars_folder, f'saloon_cars_{count}_{filename}')
    elif class_id == 1:
        output_path = os.path.join(motorcycles_folder, f'motorcycles_{count}_{filename}')
    cv2.imwrite(output_path, cropped_image)
    print(f'Saved cropped image: {output_path}')

# Initialize a counter for each class to ensure unique filenames
saloon_cars_count = 0
motorcycles_count = 0

# Process each image and its corresponding annotation to crop and save patches
for filename in os.listdir(input_folder):
    if filename.endswith('.PNG'):
        image_path = os.path.join(input_folder, filename)
        annotation_path = os.path.join(input_folder, filename.replace('.PNG', '.txt'))

        if os.path.exists(annotation_path):
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            with open(annotation_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    class_id, x_center, y_center, box_width, box_height = map(float, line.strip().split())
                    class_id = int(class_id)
                    x_center *= width
                    y_center *= height
                    box_width *= width
                    box_height *= height

                    x_min = int(x_center - (box_width / 2))
                    y_min = int(x_center - (box_width / 2))
                    x_max = int(x_center + (box_width / 2))
                    y_max = int(y_center + (box_width / 2))

                    # Crop and save the image patch
                    if class_id == 0:
                        saloon_cars_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, saloon_cars_count)
                    elif class_id == 1:
                        motorcycles_count += 1
                        crop_and_save(image, class_id, x_min, y_min, x_max, y_max, filename, motorcycles_count)

print('Cropping and saving complete.')

# Load and preprocess the dataset
dataset_new = image_dataset_from_directory(
    '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images',
    batch_size=32,
    image_size=(224, 224),  # VGG16 works with 224x224 images
    label_mode='int'
)

scaled_data = dataset_new.map(lambda x, y: (x / 255.0, y))

# Split the dataset into train and test sets
dataset_size = tf.data.experimental.cardinality(scaled_data).numpy()
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train = scaled_data.take(train_size)
test = scaled_data.skip(train_size).take(test_size)

# Define the VGG16 model architecture
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
hist = model.fit(train, validation_data=test, epochs=5)

# Model evaluation metrics
y_true = []
y_pred = []

# Evaluate the model
for batch in test.as_numpy_iterator():
    X, y = batch
    y_pred_batch = model.predict(X)
    y_true.extend(y)
    y_pred.extend(np.argmax(y_pred_batch, axis=1))

# Convert y_true and y_pred to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(f'Confusion Matrix:\n{cm}')

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()


# Calculate and print AUC, precision, recall, and F1 score
auc = roc_auc_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
classification_rep = classification_report(y_true, y_pred, target_names=class_labels)

print(f'AUC: {auc}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Classification Report:\n{classification_rep}')

# Plot training and validation accuracy
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='blue', label='val_accuracy')
fig.suptitle('ACCURACY', fontsize=20)
plt.legend(loc='upper right')
plt.show()

# Plot training and validation loss
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='blue', label='val_loss')
fig.suptitle('LOSS', fontsize=20)
plt.legend(loc='upper right')
plt.show()



**YOLOv8 Model**

In [ ]:


# Function to read YOLO annotations
def read_yolo_annotations(annotation_file, img_width, img_height):
    boxes = []
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            boxes.append((class_id, x_min, y_min, x_max, y_max))
    return boxes

# Directory paths
input_dir = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
output_dir = os.path.join(input_dir, 'images')

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all image files in the input directory
image_files = [f for f in os.listdir(input_dir) if f.endswith('.PNG')]

# Loop through each image file
for file_name in image_files:
    # Load image
    img_path = os.path.join(input_dir, file_name)
    image = cv2.imread(img_path)
    if image is not None:
        img_height, img_width = image.shape[:2]

        # Read annotations
        annotation_path = os.path.join(input_dir, file_name.replace('.PNG', '.txt'))
        if os.path.exists(annotation_path):
            boxes = read_yolo_annotations(annotation_path, img_width, img_height)

            # Draw bounding boxes and labels
            for (class_id, x_min, y_min, x_max, y_max) in boxes:
                color = (0, 255, 0)  # Green color for bounding boxes
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)
                label = "Motorcycles" if class_id == 1 else "Saloon Cars"  # Assigning class labels based on class_id
                cv2.putText(image, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Save the annotated image in the output directory
            output_path = os.path.join(output_dir, f"images_{file_name}")
            cv2.imwrite(output_path, image)

print("Annotated images with bounding boxes saved in the 'images' directory.")


Annotated images with bounding boxes saved in the 'images' directory.


In [ ]:
import os
import cv2

# Function to read YOLO annotations
def read_yolo_annotations(annotation_file, img_width, img_height):
    boxes = []
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            boxes.append((class_id, x_min, y_min, x_max, y_max))
    return boxes

# Directory paths
input_dir = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
output_annotation_dir = os.path.join(input_dir, 'labels')

# Create the output directory if it doesn't exist
if not os.path.exists(output_annotation_dir):
    os.makedirs(output_annotation_dir)

# List all image files in the input directory
image_files = [f for f in os.listdir(input_dir) if f.endswith('.PNG')]

# Loop through each image file
for file_name in image_files:
    # Read annotations
    annotation_path = os.path.join(input_dir, file_name.replace('.PNG', '.txt'))
    if os.path.exists(annotation_path):
        img = cv2.imread(os.path.join(input_dir, file_name))
        img_height, img_width = img.shape[:2]
        boxes = read_yolo_annotations(annotation_path, img_width, img_height)

        # Save annotations to annotation directory
        annotation_output_path = os.path.join(output_annotation_dir, f"{file_name.replace('.PNG', '.txt')}")
        with open(annotation_output_path, 'w') as annotation_file:
            for (class_id, x_min, y_min, x_max, y_max) in boxes:
                annotation_line = f"{class_id} {x_min} {y_min} {x_max - x_min} {y_max - y_min}\n"
                annotation_file.write(annotation_line)

print("Annotations saved in the 'label' directory.")


Annotations saved in the 'label' directory.


In [ ]:
import cv2
import os
from torchvision.transforms import functional as F

# Function to read YOLO annotations
def read_yolo_annotations(annotation_file, img_width, img_height):
    boxes = []
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)
            boxes.append((class_id, x_min, y_min, x_max, y_max))
    return boxes

# Function to load images, resize them, and normalize pixel values
def load_images_and_annotations(input_dir):
    images = []
    annotations = []

    # List all image files in the input directory
    image_files = [f for f in os.listdir(input_dir) if f.endswith('.PNG')]

    for file_name in image_files:
        img_path = os.path.join(input_dir, file_name)
        annotation_path = os.path.join(input_dir, file_name.replace('.PNG', '.txt'))

        # Read image
        image = cv2.imread(img_path)
        if image is not None:
            img_height, img_width = image.shape[:2]

            # Resize image to desired size
            image = cv2.resize(image, (416, 416))

            # Normalize pixel values
            image = F.to_tensor(image)
            image = F.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

            # Read annotations
            if os.path.exists(annotation_path):
                boxes = read_yolo_annotations(annotation_path, img_width, img_height)
                annotations.append(boxes)

            images.append(image)

    return images, annotations

# Directory path
input_dir = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'

# Load images and annotations
images, annotations = load_images_and_annotations(input_dir)

print("Images loaded:", len(images))
print("Annotations loaded:", len(annotations))


Images loaded: 2203
Annotations loaded: 441


In [ ]:
import os
import shutil
import random

# Define the directory paths
base_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
image_directory = os.path.join(base_directory, 'images')
train_image_directory = os.path.join(image_directory, 'train')
test_image_directory = os.path.join(image_directory, 'test')

# Create train and test directories if they don't exist
os.makedirs(train_image_directory, exist_ok=True)
os.makedirs(test_image_directory, exist_ok=True)

# List all annotated files (assuming annotations are in .PNG format)
annotation_files = [f for f in os.listdir(image_directory) if f.endswith('.PNG')]

# Shuffle the files
random.shuffle(annotation_files)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(annotation_files) * split_ratio)

# Split the files into training and testing sets
train_files = annotation_files[:split_index]
test_files = annotation_files[split_index:]

# Move the files to the respective directories
for file_name in train_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(train_directory, file_name)
    shutil.move(src_path, dst_path)

for file_name in test_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(test_image_directory, file_name)
    shutil.move(src_path, dst_path)

print("Annotated data split into training and testing sets.")


Annotated data split into training and testing sets.


In [ ]:
import os
import shutil
import random
# Define the directory paths
yolo_labels_directory_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
label_directory = os.path.join(yolo_labels_directory_directory, 'labels')
train_label_directory = os.path.join(label_directory, 'train')
test_label_directory = os.path.join(yolo_labels_directory_directory, 'test')

#yolo_labels_directory_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset/labels'
#train_labels_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset/train_labels'
#test_labels_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset/test_labels'

# Create train and test directories if they don't exist
os.makedirs(train_label_directory, exist_ok=True)
os.makedirs(test_label_directory, exist_ok=True)

# List all annotation files
label_files = [f for f in os.listdir(yolo_labels_directory_directory ) if f.endswith('.txt')]

# Shuffle the files
random.shuffle(label_files)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(label_files) * split_ratio)

# Split the files into training and testing sets
train_files = label_files[:split_index]
test_files = label_files[split_index:]

# Move the files to the respective directories
for file_name in train_files:
    src_path = os.path.join(yolo_labels_directory_directory , file_name)
    dst_path = os.path.join(train_label_directory, file_name)
    shutil.move(src_path, dst_path)

for file_name in test_files:
    src_path = os.path.join(yolo_labels_directory_directory , file_name)
    dst_path = os.path.join(yolo_labels_directory_directory , file_name)
    shutil.move(src_path, dst_path)

print("Annotations split into training and testing sets.")


Annotations split into training and testing sets.


In [ ]:
# Define the root directory of your YOLO dataset
dataset_root = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'

In [ ]:
# Construct the paths for images and labels directories
train_images_directory = os.path.join(dataset_root, 'images')
train_labels_directory = os.path.join(dataset_root, 'labels')

In [ ]:
import cv2
import numpy as np
import os

# Function to load images from a directory
def load_images_from_dir(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return np.array(images)

# Function to load labels from a directory
def load_labels_from_dir(directory):
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            label_path = os.path.join(directory, filename)
            with open(label_path, 'r') as file:
                lines = file.readlines()
                # Assuming each line in the label file contains the class and bounding box coordinates
                label = [[float(x) for x in line.strip().split()] for line in lines]
                labels.extend(label)  # Use extend instead of append
    return np.array(labels)


# Define the directory paths for images and labels
train_images_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset/train_images'
train_labels_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset/train_labels'

# Load training images and labels
train_images = load_images_from_dir(train_images_directory)
train_labels = load_labels_from_dir(train_labels_directory)


In [ ]:
import yaml
import os

# Define the dataset configuration
dataset_config = {
    'train': {
        'images': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images/train',
        'labels': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train'
    },
    'val': {
        'images': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images/train',
        'labels': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train'
    },
    'nc': 2,  # Number of classes
    'names': ['Motorcycles', 'Saloon Cars']  # Class names
}

# Define the directory path for the YAML file
directory = '/content/drive/MyDrive/yolo_dataset'
yaml_file = 'config_file.yaml'
yaml_file_path = os.path.join(directory, yaml_file)

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save dataset configuration to YAML file
with open(yaml_file_path, 'w') as file:
    yaml.dump(dataset_config, file)


In [ ]:
import yaml
import os

# Define YOLOv5 model configuration
yolov5_config = {
    'model': {
        'type': 'YOLOv5',  # YOLOv5 type
        'nc': 2,  # Number of classes
        'depth_multiple': 0.33,  # Model depth multiple
        'width_multiple': 0.50,  # Model width multiple
    },
    'train': {
        'yaml_file': '/content/drive/MyDrive/yolo_dataset/yolov5_config_file.yaml',  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for training
        'epochs': 10,  # Number of epochs
        'weights': 'yolov5s.pt',  # Path to initial weights file (optional)
        'img_size': [416, 416],  # Image size for training
        'device': '0',  # Device for training (GPU index)
        'multi_scale': False,  # Use multi-scale training
        'cache_images': True,  # Cache images for faster training
    },
    'val': {
        'yaml_file': '/content/drive/MyDrive/yolo_dataset/yolov5_config_file.yaml',  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for testing
        'img_size': [416, 416],  # Image size for testing
        'conf_thres': 0.001,  # Confidence threshold for testing
        'iou_thres': 0.6,  # IoU threshold for testing
        'device': '0',  # Device for testing (GPU index)
        'augment': False,  # Augment testing
    },
    'export': {
        'weights': 'yolov5s_final.pt',  # Path to exported final weights file
        'img_size': [416, 416],  # Image size for exporting
        'batch_size': 1,  # Batch size for exporting
    },
}

# Define the directory path for the YAML file
directory = '/content/drive/MyDrive/yolo_dataset'
yolov5_yaml_file = 'yolov5_config_file.yaml'
yolov5_yaml_file_path = os.path.join(directory, yolov5_yaml_file)

# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

# Save YOLOv5 model configuration to YAML file
with open(yolov5_yaml_file_path, 'w') as file:
    yaml.dump(yolov5_config, file)


In [ ]:
import yaml

# Path to dataset configuration YAML file
dataset_config_file = '/content/drive/MyDrive/yolo_dataset/config_file.yaml'
yolov5_config_file = '/content/drive/MyDrive/yolo_dataset/yolov5_config_file.yaml'

# Read dataset configuration from YAML file
with open(dataset_config_file, 'r') as file:
    dataset_config = yaml.safe_load(file)

# Read YOLOv5 configuration from YAML file
with open(yolov5_config_file, 'r') as file:
    yolov5_config = yaml.safe_load(file)


In [ ]:
import os
import yaml

# Define the dataset configuration
dataset_config = {
    'train': {
        'images': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images/train',
        'labels': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train'
    },
    'val': {
        'images': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/images/train',
        'labels': '/content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train'
    },
    'nc': 2,  # Number of classes
    'names': ['Motorcycles', 'Saloon Cars']  # Class names
}

# Specify the directory where you want to save the YAML file
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'  # Adjust this path if needed
os.makedirs(ROOT_DIR, exist_ok=True)

# Save the configuration to a YAML file
config_file_path = os.path.join(ROOT_DIR, 'data.yaml')
with open(config_file_path, 'w') as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

print(f"data.yaml file created successfully at {config_file_path}.")


data.yaml file created successfully at /content/drive/MyDrive/yolo_dataset/yolo_dataset/data.yaml.


In [ ]:
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os

from ultralytics import YOLO





In [ ]:
# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch


In [ ]:
# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "data.yaml"), epochs=1)  # train the model

Ultralytics YOLOv8.2.29 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/yolo_dataset/yolo_dataset/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache... 0 images, 1762 backgrounds, 0 corrupt: 100%|██████████| 1762/1762 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache... 0 images, 1762 backgrounds, 0 corrupt: 100%|██████████| 1762/1762 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/train20/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


...

                   all       1762          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



1 epochs completed in 0.570 hours.
Optimizer stripped from runs/detect/train20/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train20/weights/best.pt, 6.2MB

Validating runs/detect/train20/weights/best.pt...
Ultralytics YOLOv8.2.29 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [06:29<00:00,  6.96s/it]

                   all       1762          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 2.3ms preprocess, 196.6ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to runs/detect/train20


In [ ]:
!scp -r /content/runs '/content/drive/MyDrive/yolo_dataset/object_train_data'

In [ ]:
import os
import shutil
import random

# Define the directory paths
base_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
image_directory = os.path.join(base_directory, 'images')
label_directory = os.path.join(base_directory, 'labels')

train_image_directory = os.path.join(image_directory, 'train')
test_image_directory = os.path.join(image_directory, 'test')
train_label_directory = os.path.join(label_directory, 'train')
test_label_directory = os.path.join(label_directory, 'test')

# Create train and test directories if they don't exist
os.makedirs(train_image_directory, exist_ok=True)
os.makedirs(test_image_directory, exist_ok=True)
os.makedirs(train_label_directory, exist_ok=True)
os.makedirs(test_label_directory, exist_ok=True)

# List all image and label files in the input directory
image_files = [f for f in os.listdir(image_directory) if f.endswith('.PNG')]
label_files = [f for f in os.listdir(label_directory) if f.endswith('.txt')]

# Shuffle the files
random.shuffle(image_files)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

# Split the files into training and testing sets
train_image_files = image_files[:split_index]
test_image_files = image_files[split_index:]

# Move the image files to the respective directories
for file_name in train_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(train_image_directory, file_name)
    shutil.move(src_path, dst_path)

for file_name in test_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(test_image_directory, file_name)
    shutil.move(src_path, dst_path)

# Move the label files to the respective directories
for file_name in train_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path = os.path.join(train_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

for file_name in test_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path = os.path.join(test_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

print("Annotated data split into training and testing sets.")


Annotated data split into training and testing sets.


In [ ]:
# Step 1: Preparing the Dataset
import os
import shutil
import random

# Define the directory paths
base_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
image_directory = os.path.join(base_directory, 'images')
label_directory = os.path.join(base_directory, 'labels')

train_image_directory = os.path.join(image_directory, 'train')
test_image_directory = os.path.join(image_directory, 'test')
train_label_directory = os.path.join(label_directory, 'train')
test_label_directory = os.path.join(label_directory, 'test')

# Create train and test directories if they don't exist
os.makedirs(train_image_directory, exist_ok=True)
os.makedirs(test_image_directory, exist_ok=True)
os.makedirs(train_label_directory, exist_ok=True)
os.makedirs(test_label_directory, exist_ok=True)

# List all image and label files in the input directory
image_files = [f for f in os.listdir(image_directory) if f.endswith('.PNG')]
label_files = [f for f in os.listdir(label_directory) if f.endswith('.txt')]

# Shuffle the files
random.shuffle(image_files)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

# Split the files into training and testing sets
train_image_files = image_files[:split_index]
test_image_files = image_files[split_index:]

# Move the image files to the respective directories
for file_name in train_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(train_image_directory, file_name)
    shutil.move(src_path, dst_path)

for file_name in test_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(test_image_directory, file_name)
    shutil.move(src_path, dst_path)

# Move the label files to the respective directories
for file_name in train_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path =    os.path.join(train_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

for file_name in test_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path = os.path.join(test_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

print("Annotated data split into training and testing sets.")

# Step 2: Creating Configuration Files
import os
import yaml

# Define the dataset configuration
dataset_config = {
    'train': os.path.join(base_directory, 'images/train'),
    'val': os.path.join(base_directory, 'images/test'),
    'nc': 2,  # Number of classes
    'names': ['Motorcycles', 'Saloon Cars']  # Class names
}

# Specify the directory where you want to save the YAML file
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset'
os.makedirs(ROOT_DIR, exist_ok=True)

# Save the configuration to a YAML file
config_file_path = os.path.join(ROOT_DIR, 'data.yaml')
with open(config_file_path, 'w') as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

print(f"data.yaml file created successfully at {config_file_path}.")

# Define YOLOv8 model configuration
yolov8_config = {
    'model': {
        'type': 'YOLOv8',  # YOLOv8 type
        'nc': 2,  # Number of classes
        'depth_multiple': 0.33,  # Model depth multiple
        'width_multiple': 0.50,  # Model width multiple
    },
    'train': {
        'yaml_file': os.path.join(ROOT_DIR, 'data.yaml'),  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for training
        'epochs': 10,  # Number of epochs
        'weights': 'yolov8n.pt',  # Path to initial weights file (optional)
        'img_size': [416, 416],  # Image size for training
        'device': '0',  # Device for training (GPU index)
        'multi_scale': False,  # Use multi-scale training
        'cache_images': True,  # Cache images for faster training
    },
    'val': {
        'yaml_file': os.path.join(ROOT_DIR, 'data.yaml'),  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for testing
        'img_size': [416, 416],  # Image size for testing
        'conf_thres': 0.001,  # Confidence threshold for testing
        'iou_thres': 0.6,  # IoU threshold for testing
        'device': '0',  # Device for testing (GPU index)
        'augment': False,  # Augment testing
    },
    'export': {
        'weights': 'yolov8n_final.pt',  # Path to exported final weights file
        'img_size': [416, 416],  # Image size for exporting
        'batch_size': 1,  # Batch size for exporting
    },
}

# Define the directory path for the YAML file
yolov8_yaml_file = 'yolov8_config_file.yaml'
yolov8_yaml_file_path = os.path.join(ROOT_DIR, yolov8_yaml_file)

# Create the directory if it doesn't exist
os.makedirs(ROOT_DIR, exist_ok=True)

# Save YOLOv8 model configuration to YAML file
with open(yolov8_yaml_file_path, 'w') as file:
    yaml.dump(yolov8_config, file, default_flow_style=False)

print(f"yolov8_config_file.yaml file created successfully at {yolov8_yaml_file_path}.")

# Step 3: Training the YOLOv8 Model
!pip install ultralytics

import os
from ultralytics import YOLO

# Define the paths
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset'
data_yaml_path = os.path.join(ROOT_DIR, 'data.yaml')

# Load a YOLOv8 model
model = YOLO("yolov8n.yaml")  # you can also specify a pre-trained model here

# Train the model
results = model.train(data=data_yaml_path, epochs=10, imgsz=416, batch=16)

# Evaluate the model
metrics = model.val(data=data_yaml_path, imgsz=416, batch=16)

print("Training and evaluation completed.")
print("Results:", metrics)



Annotated data split into training and testing sets.
data.yaml file created successfully at /content/drive/MyDrive/yolo_dataset/data.yaml.
yolov8_config_file.yaml file created successfully at /content/drive/MyDrive/yolo_dataset/yolov8_config_file.yaml.
Ultralytics YOLOv8.2.30 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/yolo_dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=

train: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train... 2 images, 1830 backgrounds, 0 corrupt: 100%|██████████| 1832/1832 [00:39<00:00, 45.88it/s] 


train: New cache created: /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/test... 0 images, 89 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<00:00, 174.78it/s]

val: WARNING ⚠️ No labels found in /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/test.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

       3/10         0G      0.106      9.347     0.0727          0        416: 100%|██████████| 115/115 [12:17<00:00,  6.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:11<00:00,  3.84s/it]

                   all         89          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/10         0G    0.07433      3.219    0.04641          0        416: 100%|██████████| 115/115 [12:34<00:00,  6.57s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:10<00:00,  3.34s/it]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
# Step 1: Preparing the Dataset
import os
import shutil
import random

# Define the directory paths
base_directory = '/content/drive/MyDrive/yolo_dataset/yolo_dataset'
image_directory = os.path.join(base_directory, 'images')
label_directory = os.path.join(base_directory, 'labels')

train_image_directory = os.path.join(image_directory, 'train')
test_image_directory = os.path.join(image_directory, 'test')
train_label_directory = os.path.join(label_directory, 'train')
test_label_directory = os.path.join(label_directory, 'test')

# Create train and test directories if they don't exist
os.makedirs(train_image_directory, exist_ok=True)
os.makedirs(test_image_directory, exist_ok=True)
os.makedirs(train_label_directory, exist_ok=True)
os.makedirs(test_label_directory, exist_ok=True)

# List all image and label files in the input directory
image_files = [f for f in os.listdir(image_directory) if f.endswith('.PNG')]
label_files = [f for f in os.listdir(label_directory) if f.endswith('.txt')]

# Shuffle the files
random.shuffle(image_files)

# Define the split ratio (e.g., 80% train, 20% test)
split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

# Split the files into training and testing sets
train_image_files = image_files[:split_index]
test_image_files = image_files[split_index:]

# Move the image files to the respective directories
for file_name in train_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(train_image_directory, file_name)
    shutil.move(src_path, dst_path)

for file_name in test_image_files:
    src_path = os.path.join(image_directory, file_name)
    dst_path = os.path.join(test_image_directory, file_name)
    shutil.move(src_path, dst_path)

# Move the label files to the respective directories
for file_name in train_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path =    os.path.join(train_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

for file_name in test_image_files:
    label_name = file_name.replace('.PNG', '.txt')
    src_path = os.path.join(label_directory, label_name)
    dst_path = os.path.join(test_label_directory, label_name)
    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

print("Annotated data split into training and testing sets.")

# Step 2: Creating Configuration Files
import os
import yaml

# Define the dataset configuration
dataset_config = {
    'train': os.path.join(base_directory, 'images/train'),
    'val': os.path.join(base_directory, 'images/test'),
    'nc': 2,  # Number of classes
    'names': ['Motorcycles', 'Saloon Cars']  # Class names
}

# Specify the directory where you want to save the YAML file
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset'
os.makedirs(ROOT_DIR, exist_ok=True)

# Save the configuration to a YAML file
config_file_path = os.path.join(ROOT_DIR, 'data.yaml')
with open(config_file_path, 'w') as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

print(f"data.yaml file created successfully at {config_file_path}.")

# Define YOLOv8 model configuration
yolov8_config = {
    'model': {
        'type': 'YOLOv8',  # YOLOv8 type
        'nc': 2,  # Number of classes
        'depth_multiple': 0.33,  # Model depth multiple
        'width_multiple': 0.50,  # Model width multiple
    },
    'train': {
        'yaml_file': os.path.join(ROOT_DIR, 'data.yaml'),  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for training
        'epochs': 4,  # Number of epochs
        'weights': 'yolov8n.pt',  # Path to initial weights file (optional)
        'img_size': [416, 416],  # Image size for training
        'device': '0',  # Device for training (GPU index)
        'multi_scale': False,  # Use multi-scale training
        'cache_images': True,  # Cache images for faster training
    },
    'val': {
        'yaml_file': os.path.join(ROOT_DIR, 'data.yaml'),  # Path to dataset YAML file
        'batch_size': 16,  # Batch size for testing
        'img_size': [416, 416],  # Image size for testing
        'conf_thres': 0.001,  # Confidence threshold for testing
        'iou_thres': 0.6,  # IoU threshold for testing
        'device': '0',  # Device for testing (GPU index)
        'augment': False,  # Augment testing
    },
    'export': {
        'weights': 'yolov8n_final.pt',  # Path to exported final weights file
        'img_size': [416, 416],  # Image size for exporting
        'batch_size': 1,  # Batch size for exporting
    },
}

# Define the directory path for the YAML file
yolov8_yaml_file = 'yolov8_config_file.yaml'
yolov8_yaml_file_path = os.path.join(ROOT_DIR, yolov8_yaml_file)

# Create the directory if it doesn't exist
os.makedirs(ROOT_DIR, exist_ok=True)

# Save YOLOv8 model configuration to YAML file
with open(yolov8_yaml_file_path, 'w') as file:
    yaml.dump(yolov8_config, file, default_flow_style=False)

print(f"yolov8_config_file.yaml file created successfully at {yolov8_yaml_file_path}.")

# Step 3: Training the YOLOv8 Model
!pip install ultralytics

import os
from ultralytics import YOLO

# Define the paths
ROOT_DIR = '/content/drive/MyDrive/yolo_dataset'
data_yaml_path = os.path.join(ROOT_DIR, 'data.yaml')

# Load a YOLOv8 model
model = YOLO("yolov8n.yaml")  # you can also specify a pre-trained model here

# Train the model
results = model.train(data=data_yaml_path, epochs=4, imgsz=416, batch=16)

# Evaluate the model
metrics = model.val(data=data_yaml_path, imgsz=416, batch=16)

print("Training and evaluation completed.")
print("Results:", metrics)


Annotated data split into training and testing sets.
data.yaml file created successfully at /content/drive/MyDrive/yolo_dataset/data.yaml.
yolov8_config_file.yaml file created successfully at /content/drive/MyDrive/yolo_dataset/yolov8_config_file.yaml.
Ultralytics YOLOv8.2.30 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/yolo_dataset/data.yaml, epochs=4, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=T

train: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/train.cache... 2 images, 1830 backgrounds, 0 corrupt: 100%|██████████| 1832/1832 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/test.cache... 0 images, 89 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

WARNING ⚠️ No labels found in /content/drive/MyDrive/yolo_dataset/yolo_dataset/labels/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.96s/it]

                   all         89          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels


Speed: 1.2ms preprocess, 106.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train42
Training and evaluation completed.
Results: 

AttributeError: 'DetMetrics' object has no attribute 'curves_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass